In [1]:
import pandas as pd
import numpy as np

In [ ]:
#REG_YYMM : 년월
#CARD_SIDO_NM : 카드이용지역_시도 (가맹점 주소 기준)
#CARD_CCG_NM : 카드이용지역_시군구 (가맹점 주소 기준)
#STD_CLSS_NM : 업종명
#HOM_SIDO_NM : 거주지역_시도, (고객 집주소 기준)
#HOM_CCG_NM : 거주지역_시군구 (고객 집주소 기준)
#AGE : 연령대
#SEX_CTGO_CD : 성별 (1: 남성, 2: 여성)
#FLC : 가구생애주기 (1: 1인가구, 2: 영유아자녀가구, 3: 중고생자녀가구, 4: 성인자녀가구, 5: 노년가구)
#CSTMR_CNT : 이용고객수 (명)
#AMT : 이용금액 (원)
#CNT : 이용건수 (건)

In [ ]:
#data = pd.read_csv('201901-202003.csv')
#data.head()
#data.shape

In [ ]:
#data.to_pickle('201901-202003.pkl')

In [2]:
data = pd.read_csv('201901-202003.csv')

In [3]:
# 날짜 처리
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [4]:
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [5]:
df = data.copy()

# 20년 데이터(4월 포함)

In [6]:
data04 = pd.read_csv('202004.csv')

In [7]:
data04 = data04.fillna('')
data04['year'] = data04['REG_YYMM'].apply(lambda x: grap_year(x))
data04['month'] = data04['REG_YYMM'].apply(lambda x: grap_month(x))
data04 = data04.drop(['REG_YYMM'], axis=1)

In [8]:
df_2020 = df[df['year']==2020]
df_2020 = pd.concat([df_2020,data04], axis=0)
df_2020

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
20425415,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,2,1,3,345000,3,2020,1
20425416,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,3,1903450,3,2020,1
20425417,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,14,1520500,15,2020,1
20425418,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,9,1239200,9,2020,1
20425419,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,606700,4,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350317,충북,충주시,휴양콘도 운영업,충북,충주시,20s,1,1,5,77000,5,2020,4
1350318,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,6,92000,6,2020,4
1350319,충북,충주시,휴양콘도 운영업,충북,충주시,40s,2,3,5,193000,5,2020,4
1350320,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,5,86000,7,2020,4


# 데이터셋에서 연도, 월 별 비교

In [ ]:
# 연도, 월 별 '이용금액'과 '이용건수'
def month(a, b):
    data1 = df[df['year']==a]
    c = data1[data1['month']==b]
    d = c.groupby(['year','month','STD_CLSS_NM']).agg({'AMT':'sum','CNT':'sum'})
    e = d.reset_index()
    return e

# 전 월 대비 '이용금액'과 '이용건수'의 변동률
# AMT_dif, CNT_dif : 전 월 대비 변화량
def dif_month(x, y):
    z = y['AMT'] - x['AMT']
    f = y['CNT'] - x['CNT']
    c = pd.DataFrame(z)
    b = pd.DataFrame(f)
    c = c.rename(columns={'AMT':'AMT_dif'})
    b = b.rename(columns={'CNT':'CNT_dif'})
    a = x.merge(c, left_index=True, right_index=True,  how='right')
    a = a.merge(b, left_index=True, right_index=True,  how='right')
    a = a[['year','month','STD_CLSS_NM','AMT','AMT_dif','CNT','CNT_dif']]
    return a.sort_values(ascending=False, by='AMT')

# 년도 별 '이용금액'과 '이용건수'의 변화량의 차이
# AMT_dif, CNT_dif : 전 년도 대비 변화량의 차이
def dif_year(x, y):
    x = x.drop(['AMT','CNT'], axis=1)
    z = y['AMT_dif'] - x['AMT_dif']
    f = y['CNT_dif'] - x['CNT_dif']
    c = pd.DataFrame(z)
    b = pd.DataFrame(f)
    x = x.rename(columns={'AMT_dif':'AMT'})
    x = x.rename(columns={'CNT_dif':'CNT'})
    a = x.merge(c, left_index=True, right_index=True,  how='right')
    a = a.merge(b, left_index=True, right_index=True,  how='right')
    a = a[['year','month','STD_CLSS_NM','AMT','AMT_dif','CNT','CNT_dif']]
    return a.sort_values(ascending=False, by='AMT') 

In [ ]:
# 연도, 월 별 '이용금액'과 '이용건수'
def month(a, b):
    data1 = df[df['year']==a]
    c = data1[data1['month']==b]
    d = c.groupby(['year','month','STD_CLSS_NM']).agg({'AMT':'sum','CNT':'sum'})
    e = d.reset_index()
    return e

# 전 월 대비 '이용금액'과 '이용건수'의 변동률
# AMT_dif, CNT_dif : 전 월 대비 변동률
def dif_month(x, y):
    def month(x, y):
        data1 = df[df['year']==x]
        c = data1[data1['month']==y]
        d = c.groupby(['year','month','STD_CLSS_NM']).agg({'AMT':'sum','CNT':'sum'})
        p = d.reset_index()
        return p
    i = y-1
    e = month(x, y)
    o = month(x, i)
    c = pd.DataFrame((e['AMT'] - o['AMT'])/o['AMT']*100)
    b = pd.DataFrame((e['CNT'] - o['CNT'])/o['CNT']*100)
    c = c.rename(columns={'AMT':'AMT_dif'})
    b = b.rename(columns={'CNT':'CNT_dif'})
    a = e.merge(c, left_index=True, right_index=True,  how='right')
    a = a.merge(b, left_index=True, right_index=True,  how='right')
    a = a[['year','month','STD_CLSS_NM','AMT','AMT_dif','CNT','CNT_dif']]
    return a.sort_values(ascending=False, by='AMT')

# 년도 별 '이용금액'과 '이용건수'의 변화량의 차이
# AMT_dif, CNT_dif : 전 년도 대비 변화량의 차이
def dif_year(x, y):
    x = x.drop(['AMT','CNT'], axis=1)
    z = y['AMT_dif'] - x['AMT_dif']
    f = y['CNT_dif'] - x['CNT_dif']
    c = pd.DataFrame(z)
    b = pd.DataFrame(f)
    x = x.rename(columns={'AMT_dif':'AMT'})
    x = x.rename(columns={'CNT_dif':'CNT'})
    a = x.merge(c, left_index=True, right_index=True,  how='right')
    a = a.merge(b, left_index=True, right_index=True,  how='right')
    a = a[['year','month','STD_CLSS_NM','AMT','AMT_dif','CNT','CNT_dif']]
    return a.sort_values(ascending=False, by='AMT') 

# 업종 별 변동률 EDA

## q : 2019년

In [ ]:
df_q = df.groupby(['year','month','STD_CLSS_NM']).agg({'AMT':'sum'})
df_q = df_q.loc[2019]
df_q

In [ ]:
q = pd.DataFrame()
for month in range(2, 13):
    sample = (df_q.loc[month] - df_q.loc[month-1])/df_q.loc[month-1]*100
    sample['date']=month
    q = pd.concat([q,sample],axis=0)
q

## r : 2020년

In [ ]:
df_r = df_2020.groupby(['year','month','STD_CLSS_NM']).agg({'AMT':'sum'})
df_r = df_r.loc[2020]
df_r

In [ ]:
r = pd.DataFrame()
for month in range(2, 5):
    sample = (df_r.loc[month] - df_r.loc[month-1])/df_r.loc[month-1]*100
    sample['date']=month
    r = pd.concat([r,sample],axis=0)
r

# 시각화

In [ ]:
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline

plt.rcParams['font.family'] = 'NanumGothic'

In [ ]:
import matplotlib.font_manager as fm

path = 'C:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()

plt.rc('font', family=font_name)

In [ ]:
f, axes = plt.subplots(21, 2, figsize=(15, 15))
i=0
j=0
for CLSS in q.index.unique():
    w = q.loc[CLSS]
    r_w = r.loc[CLSS]
    sns.lineplot(x=w.date,y=w.AMT, ax=axes[i, j]).set_title(CLSS)
    sns.lineplot(x=r.date, y= r.AMT, ax=axes[i,j]).set_title(CLSS)
    i+=1
    if i>=21:
        i=0
        j=1
plt.show()

# AMT에 MinMax 적용해서 본다

In [ ]:
df = pd.concat([data,data04], axis=0)

In [ ]:
df_g = df.groupby(['REG_YYMM','STD_CLSS_NM']).sum()

df_g = df_g[['AMT','CNT']]

df_g.reset_index(inplace=True)
df_g.set_index('STD_CLSS_NM',inplace=True)

df_g['hue'] = df_g['REG_YYMM']
df_g['hue'] = df_g['hue'].apply(lambda x : int(str(x)[:4]))


f, axes = plt.subplots(21, 2, figsize=(50, 50),sharex=True)
i=0
j=0
for CLSS in df_g.index.unique():
    df_s = df_g.loc[CLSS]
    df_s['REG_YYMM'] =list(range(1,13)) +[1,2,3,4]
    df_s['AMT'] = (df_s['AMT'] - df_s['AMT'].min())/(df_s['AMT'].max() - df_s['AMT'].min())
    g = sns.lineplot(x= 'REG_YYMM',y='AMT', style='hue' ,data=df_s, ax=axes[i, j])
    g.set_title(CLSS)
    g.set_xticks(range(len(df_s)))
    i +=1
    if i==21:
        i=0
        j=1


f, axes = plt.subplots(21, 2, figsize=(50, 50),sharex=True)
i=0
j=0
for CLSS in df_g.index.unique():
    df_s = df_g.loc[CLSS]
    df_s['REG_YYMM'] =list(range(1,13)) +[1,2,3,4]
    g = sns.lineplot(x= 'REG_YYMM',y='CNT', style='hue' ,data=df_s, ax=axes[i, j])
    g.set_title(CLSS)
    g.set_xticks(range(len(df_s)))
    i +=1
    if i==21:
        i=0
        j=1

In [ ]:
a = dif_month(2019, 3)
a

In [ ]:
b = dif_month(2020, 3)
b

In [ ]:
a["dif"] = b["AMT_dif"] - a["AMT_dif"]
a

# 19년 4월과 20년 4월의 차이

In [ ]:
a.sort_values('dif')

In [ ]:
b = month(2020,1)
c = month(2020,2)

In [ ]:
a_20_2 = month(2020, 2)
a_20_3 = month(2020, 3)

In [ ]:
a_19_2 = month(2019, 2)
a_19_3 = month(2019, 3)

In [ ]:
# 19년 2월 3월
d_19_03 = dif_month(a_19_2, a_19_3)
d_19_03

In [ ]:
# 20년 2월 3월
d_20_03 = dif_month(a_20_2, a_20_3)
d_20_03

In [ ]:
# 월별 차이의 연도별 차이
dif_year(d_19_03, d_20_03)

# 업종별 변동율 기준으로 군집을 나눈다

In [ ]:
# 군집1 : 영향을 +로 받은 업종
# 군집2 : 영향을 -로 받은 업종
# 군집3 : 영향을 받지 않은 업종
# 군집4 : 사라진 업종
k-means 고고

# 업종별 상관관계를 본다

In [ ]:
df

In [ ]:
seoul = df[df['CARD_SIDO_NM']=='서울']
seoul

In [ ]:
# a = df.groupby('CARD_SIDO_NM')['STD_CLSS_NM'].value_counts().to_frame().reset_index()
# Sido_Clss = 지역별 업종 value_counts
Sido_Clss = df.groupby('CARD_SIDO_NM')['STD_CLSS_NM'].value_counts().to_frame()
Sido_Clss.columns=['STD_CLSS_NM_C']
Sido_Clss = Sido_Clss.reset_index()

In [ ]:
Sido_Clss

In [ ]:
서비스업 = Sido_Clss[Sido_Clss['STD_CLSS_NM']=='그외 기타 분류안된 오락관련 서비스업']
서비스업

In [ ]:
서울 = Sido_Clss[Sido_Clss['CARD_SIDO_NM']=='서울']
서울

# 지역별 업종 유무 탐색

In [ ]:
df_g = df.groupby(['year','month','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM']).count()
CS_04 = df_g.loc[2019,4].reset_index()[['CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM']]
CS4 = set(zip(CS_04['CARD_SIDO_NM'],CS_04['STD_CLSS_NM'],CS_04['HOM_SIDO_NM']))

CS_03 = df_g.loc[2020,3].reset_index()[['CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM']]
CS3 = set(zip(CS_03['CARD_SIDO_NM'],CS_03['STD_CLSS_NM'],CS_03['HOM_SIDO_NM']))

CS = CS3 & CS4

In [ ]:
temp = []
for CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM in CS3:
    for AGE in AGEs:
        for SEX_CTGO_CD in SEX_CTGO_CDs:
            for FLC in FLCs:
                for year in years:
                    for month in months:
                        temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])

# 지역별로 업종간 차이가 있으므로 업종의 차이를 두어 지역별로 예측 해본다.

# 시군구 Null 값 있는거는 세종시뿐

In [ ]:
#data[data['CARD_CCG_NM'].isnull()]['CARD_SIDO_NM'].unique()

In [ ]:
df['CARD_CCG_NM']=df['CARD_CCG_NM'].fillna('세종시')

In [ ]:
df.isnull().sum()

In [ ]:
df[df['HOM_CCG_NM'].isnull()].HOM_SIDO_NM.unique()

In [ ]:
df['HOM_CCG_NM']=data['HOM_CCG_NM'].fillna('세종시')

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# AGE에 's' 지우기
df['AGE']=df['AGE'].apply(lambda x:int(str(x)[:2]))

In [ ]:
df.info()

In [ ]:
#날짜 
data['REG_YY']=data['REG_YYMM'].apply(lambda x:int(str(x)[:4]))
data['REG_MM']=data['REG_YYMM'].apply(lambda x:int(str(x)[4:]))

In [ ]:
#날짜2
#data['REG_YY'] = (data.REG_YYMM / 100).astype(int)
#data['REG_MM'] = data.REG_YYMM % 100

In [ ]:
#가설1 : 카드사용지역(시도) = 거주지(현지인) / 카드사용지역(시도) = 거주인(외지인)
data['NATURALSI'] = np.where(data['CARD_SIDO_NM']==data['HOM_SIDO_NM'],1,0)

In [ ]:
data.sample(10)

In [ ]:
data['STD_CLSS'] = pd.Categorical(data['STD_CLSS_NM'])

In [ ]:
data['STD_CLSS'] = data['STD_CLSS'].cat.codes

In [ ]:
a = pd.DataFrame(data['STD_CLSS_NM'].drop_duplicates())

In [ ]:
b = pd.DataFrame(data['STD_CLSS'].drop_duplicates())

In [ ]:
yes = a.join(b, how='outer')
yes.sort_values(['STD_CLSS'], ascending=True)

In [ ]:
data.info()

In [ ]:
data['CARD_SIDO_NM'].unique()

In [ ]:
import sys

mod = sys.modules[__name__]
for idx in range(len(data['CARD_SIDO_NM'].unique())) :
    setattr(mod, 'card_{}'.format(data['CARD_SIDO_NM'].unique()[idx]), 
            data[data['CARD_SIDO_NM']==data['CARD_SIDO_NM'].unique()[idx]])

In [ ]:
df = data.copy()